In [ ]:
import os
import numpy as np
import redis
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Connect to Redis
redis_client = redis.Redis(host='localhost', port=6379, db=0)

In [ ]:
info = redis_client.execute_command("FT.INFO", "rag_docs")
info_dict = dict(zip(info[::2], info[1::2]))
print("Number of vectors:", info_dict.get(b'num_docs'))

In [ ]:
def embed_text(text: str) -> list:
    """Get embedding for text using OpenAI API."""
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    resp = client.embeddings.create(input=[text], model="text-embedding-3-small")
    return np.array(resp.data[0].embedding, dtype=np.float32)


def query_vector_db(query, k=3, index_name="rag_docs"):
    embedding = embed_text(query)
    query_vec = embedding.tobytes()
    result = redis_client.execute_command(
        "FT.SEARCH", index_name,
        f"*=>[KNN {k} @embedding $vec as score]",
        "PARAMS", "2", "vec", query_vec,
        "SORTBY", "score", "ASC",
        "RETURN", "2", "text", "score",
        "DIALECT", "2",
        "LIMIT", "0", str(k))
    docs = []
    for i in range(1, len(result), 2):
        doc = result[i+1]
        docs.append({"score": doc[1].decode(), "text": doc[3].decode()})
    return docs

In [ ]:
query = "How do I set a key with an expiry in Redis?"
results = query_vector_db(query, k=3)
for i, doc in enumerate(results):
    print(f"Result {i}:")
    print(f"Score: {doc['score']}")
    print(doc['text'][:50])
    print("-" * 40)
    

In [ ]:
result = redis_client.execute_command('FT.SEARCH', 'rag_docs', '*', 'RETURN', '1', 'text', 'LIMIT', '0', '5')
for i in range(2, len(result), 2):
    doc = result[i+1]
    print(doc[1])
    print("="*60)